<a href="https://colab.research.google.com/github/leetung0718/thesis/blob/main/thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# What version of Python do you have?
import sys

import tensorflow.keras
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Numpy {np.__version__}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Linux-5.10.147+-x86_64-with-glibc2.29
Tensor Flow Version: 2.11.0
Keras Version: 2.11.0

Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Numpy 1.22.4
Pandas 1.3.5
Scikit-Learn 1.0.2
GPU is NOT AVAILABLE


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/thesis/TAIEX.csv", parse_dates=True, index_col=0)
df['Return'] = df['Close'].pct_change(periods=1)
df

,Open,High,Low,Close,Volume,Return
date,,,,,,
1999-01-05,6310.41,6310.41,6111.64,6152.43,1377116000,NaN
1999-01-06,6082.02,6280.93,5988.06,6199.91,1999194031,0.007717
1999-01-07,6280.38,6409.55,6181.62,6404.31,2631142753,0.032968
1999-01-08,6371.34,6492.87,6371.34,6421.75,2864094997,0.002723
1999-01-11,6472.02,6492.90,6392.49,6406.99,2180092599,-0.002298
...,...,...,...,...,...,...
2022-12-26,14271.20,14301.11,14253.20,14285.13,3106471398,0.000946
2022-12-27,14310.19,14410.24,14310.19,14328.43,3659061689,0.003031
2022-12-28,14249.83,14249.83,14119.30,14173.10,4430499324,-0.010841


In [8]:
# YT
dataset = df.dropna().values
dataset = dataset.astype('float32')
# dataset
train_size = int(len(dataset)*0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

def to_seq(dataset, size=1):
    x, y = [], []

    for i in range(len(dataset)-size):
        window = dataset[i:(i+size), 0]
        x.append(window)
        y.append(dataset[i+size, 0])

    return np.array(x), np.array(y)

trainX, trainY = to_seq(train, size=5)
testX, testY = to_seq(test, size=5)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


In [9]:
import math
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, ConvLSTM2D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [10]:
model = Sequential()
model.add(LSTM(64, input_shape=(None, 5)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                17920     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 20,033
Trainable params: 20,033
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(trainX, trainY, validation_data=(testX, testY), verbose=2, epochs=10)

Epoch 1/10
131/131 - 5s - loss: 52766880.0000 - val_loss: 152201808.0000 - 5s/epoch - 37ms/step
Epoch 2/10
131/131 - 1s - loss: 52221776.0000 - val_loss: 150839728.0000 - 510ms/epoch - 4ms/step
Epoch 3/10
131/131 - 1s - loss: 51156936.0000 - val_loss: 148592080.0000 - 511ms/epoch - 4ms/step
Epoch 4/10
131/131 - 0s - loss: 49598368.0000 - val_loss: 145532304.0000 - 490ms/epoch - 4ms/step
Epoch 5/10
131/131 - 1s - loss: 47604088.0000 - val_loss: 141751936.0000 - 530ms/epoch - 4ms/step
Epoch 6/10
131/131 - 0s - loss: 45241584.0000 - val_loss: 137342640.0000 - 474ms/epoch - 4ms/step
Epoch 7/10
131/131 - 0s - loss: 42580228.0000 - val_loss: 132411856.0000 - 491ms/epoch - 4ms/step
Epoch 8/10
131/131 - 1s - loss: 39688264.0000 - val_loss: 127043184.0000 - 523ms/epoch - 4ms/step
Epoch 9/10
131/131 - 0s - loss: 36636944.0000 - val_loss: 121346792.0000 - 474ms/epoch - 4ms/step
Epoch 10/10
131/131 - 0s - loss: 33493404.0000 - val_loss: 115411824.0000 - 488ms/epoch - 4ms/step
